In [9]:
import numpy as np
import pandas as pd
import os

#feature extraction modules
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD, SparsePCA, PCA
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import StandardScaler

#confusion matrix
from sklearn.metrics import confusion_matrix

folder = os.getcwd() ; print folder

/home/arda/Documents/DSG


In [2]:
#pd_train = pd.io.pickle.read_pickle(folder + '/data_munged/pd_train_tagged')
#pd_test = pd.io.pickle.read_pickle(folder + '/data_munged/pd_test_tagged')

pd_train = pd.io.pickle.read_pickle(folder + '/data_munged/pd_train_munged')
pd_test = pd.io.pickle.read_pickle(folder + '/data_munged/pd_test_munged')

In [3]:
pd_train['Sentence1'] = pd_train.apply(lambda x:str(x['Sentence'])+' '+str(int(x['nb_caps_word']))+' '+
                                  str(int(x['nb_!']))+' '+str(int(x['nb_,;'])),axis=1)

pd_train['Sentence2'] = pd_train.apply(lambda x:str(x['Sentence'])+' '+str(int(x['nb_caps_char']))+' '+
                                  str(int(x['nb_!']))+' '+str(int(x['nb_,;'])),axis=1)

pd_test['Sentence1'] = pd_test.apply(lambda x:str(x['Sentence'])+' '+str(int(x['nb_caps_word']))+' '+
                                  str(int(x['nb_!']))+' '+str(int(x['nb_,;'])),axis=1)

pd_test['Sentence2'] = pd_test.apply(lambda x:str(x['Sentence'])+' '+str(int(x['nb_caps_char']))+' '+
                                  str(int(x['nb_!']))+' '+str(int(x['nb_,;'])),axis=1)

In [16]:
list_pattern = ['\\w{1,}','(?u)\\b\\w+\\b']
count = CountVectorizer(lowercase=False, stop_words=None, token_pattern=list_pattern[0],
                        ngram_range=(1, 2), analyzer=u'word',max_df=1.0, min_df=2, 
                        max_features=None,vocabulary=None, binary=False)

tfidf = TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True)

StdScl = StandardScaler(copy=True, with_mean=False, with_std=False)

class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key].values

In [19]:
transormData = Pipeline([
        ('selector', ColumnSelector(key='Sentence_stemmed')),
        ('count', count),
        ('tfid' , tfidf),
        #('StdScl' , StdScl)
    ])

Y = pd_train['Author']
X = transormData.fit_transform(pd_train,Y) ; print X.shape
X_test = transormData.transform(pd_test)

(28723, 63447)


In [20]:
params = np.arange(1, 4, 0.2 ) ; 
#params = [50,60, 70, 80, 90, 100,]
#params = [60]
params = np.arange(0.1, 1, 0.1 ) ; 
print params

results=[]
for i in params:
    #sklearn cross validation
    ridge = RidgeClassifier(alpha=i, fit_intercept=True, normalize=False, copy_X=False,
                max_iter=None, tol=0.001, class_weight=None, solver='sparse_cg')
    
    clf = ridge

    scores = cross_val_score(clf, X,Y,scoring='accuracy',cv=5, n_jobs=1)
    
    print ("Accuracy: %0.4f (+/- %0.4f), alpha: %s" % (scores.mean(), scores.std() * 2, i))

[ 0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9]
Accuracy: 0.7323 (+/- 0.0704), alpha: 0.1
Accuracy: 0.7451 (+/- 0.0698), alpha: 0.2
Accuracy: 0.7524 (+/- 0.0679), alpha: 0.3
Accuracy: 0.7553 (+/- 0.0691), alpha: 0.4
Accuracy: 0.7578 (+/- 0.0711), alpha: 0.5
Accuracy: 0.7593 (+/- 0.0734), alpha: 0.6
Accuracy: 0.7594 (+/- 0.0740), alpha: 0.7
Accuracy: 0.7604 (+/- 0.0744), alpha: 0.8
Accuracy: 0.7605 (+/- 0.0752), alpha: 0.9


In [ ]:
#best ridge Accuracy: 0.7606 (+/- 0.0687), alpha: 0.6 = 86,6084
#OR
#Accuracy: 0.7644 (+/- 0.0745), alpha: 0.6 == 86,86034%.

In [46]:
alpha = .6
for i in range(2):
    ridge = RidgeClassifier(alpha=alpha, fit_intercept=True, normalize=False, copy_X=False,
                max_iter=None, tol=0.001, class_weight=None, solver='sparse_cg')
       
    clf = ridge

    scores=[]
    confusion_mat = np.zeros([6,6],dtype=int)
    for cv in range(5):
        x_train, x_val, y_train, y_val = train_test_split(X,Y, test_size=0.2)
        
        clf.fit(x_train,y_train)
        
        scores.append(clf.score(x_val,y_val))
        confusion_mat+=confusion_matrix(clf.predict(x_val),y_val)

    print ("Accuracy: %0.4f (+/- %0.4f), alpha: %s" % (np.mean(scores), np.mean(scores) * 2, i))
    
print clf.classes_
print confusion_mat

Accuracy: 0.8010 (+/- 1.6021), alpha: 0
Accuracy: 0.7997 (+/- 1.5994), alpha: 1
['austen' 'doyle' 'poe' 'shakespeare' 'twain' 'wilde']
[[9048  701   46  156  403  878]
 [ 320 4035  122   96  389  453]
 [   0    0   35    0    2    0]
 [  14   11    3 1762   83   28]
 [ 172  436   30  112 5037  329]
 [ 347  368   17   51  186 3055]]


# SUBMIT KAGGLE

In [47]:
clf.fit(X,Y)
y_pred = clf.predict(X_test)

In [48]:
submit = pd.DataFrame(index=None)
submit['Id']=pd_test['Id']
submit['Pred']=y_pred

In [49]:
submit.to_csv(folder+'/results/0.arda1.csv',sep=';',index=None)